<img align="left" src="https://www.taborelec.com/Pictures/logo.png">

# IQM HALF

### Description:
This script comes to demonstrate .....


### set up:


### Proteus version: 
ALICE


# IQM HALF

This script will test "IQM HALF" mode. In this mode two segments are generated : sin and cos.

When a sin signal is modulated with a cos signal, only one sideband signal relative to the LO signal

Should be present.

Connect channel 1 to a spectrum anlayzer at center frequency of 100MHz.

In [1]:
def getIntepolation(i):
    switcher={
                1:'X1',
                2:'X2',
                4:'X4',
                8:'X8'
             }
    return switcher.get(i,"Invalid Interpolation")

def getMaxRate(i):
    switcher={
                'X1': 2.5e9,
                'X2': 5.0e9,
                'X4': 9.0e9,
                'X8': 9.0e9,
             }
    return switcher.get(i,"2500")

# select here the Interpolation mode : 1,2,4 or 8
Interpolation = getIntepolation(1)
maxRate = getMaxRate(Interpolation)

        
print('Interpolation = ',Interpolation)
print('maxRate = ',maxRate)

Interpolation =  X1
maxRate =  2500000000.0


In [2]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

In [3]:
import os
import gc
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst

### Before we start

This example is designed to work both when there is a LAN based communication-service (usually on benchtop/desktop)<br>
And when there's no communication-service, and the user's application has access to hardware-modules in the PXI chassis.

In the first case, when there's a LAN communication, this example uses the class `TEVisaInst` from `tevisainst`.<br>

In the second case, this example uses the classes `TEProteusAdmin` and `TEProteusInst` from `teproteus`.

In [4]:
# Connect to instrument

inst = None
admin = None

conn_msg_str = ''

def disconnect():
    global inst
    global admin
    if inst is not None:
        try:
            inst.close_instrument()            
        except:
            pass
        inst = None
    if admin is not None:
        try:
            admin.close_inst_admin()
        except:
            pass
        admin = None
    gc.collect()

def connect_to_pxi_slot(slot_id):
    global inst
    global admin
    try:
        disconnect()        
        admin = TEProteusAdmin()
        admin.open_inst_admin()        
        inst = admin.open_instrument(slot_id, reset_hot_flag=True)
    except:
        pass

def connect_to_lan_server(ip_address):
    global inst
    try:
        disconnect()
        inst = TEVisaInst(ip_address, port=5025)
    except:
        pass

def connect(ipaddr_or_slotid):
    try:
        disconnect()
        if isinstance(ipaddr_or_slotid, str) and '.' in ipaddr_or_slotid:
            connect_to_lan_server(ipaddr_or_slotid)
        else:
            connect_to_pxi_slot(ipaddr_or_slotid)
    except:
        pass

addr_textbox = widgets.Text(
    value='',
    placeholder='IP Address or Slot Id',
    description='Address:',
    disabled=False
)

output = widgets.Output()

def print_conn_message(msg_str):
    global output
    global conn_msg_str
    with output:
        if len(conn_msg_str) > 0:
            print('\r', end='')
            print(' ' * len(conn_msg_str), end='')
            print('\r', end='')
        conn_msg_str = str(msg_str)
        if len(conn_msg_str) > 0:
            print(conn_msg_str, end='')

def on_button_connect_clicked(b):
    global inst
    global addr_textbox
    print_conn_message('')
    disconnect()
    if addr_textbox is not None:
        conn_str = addr_textbox.value
        if conn_str:
            print_conn_message('Connecting to {0} ..'.format(conn_str))
            connect(conn_str)
            print_conn_message('')
    if inst is not None:
        idn_str = inst.send_scpi_query('*IDN?')
        if idn_str:
            print_conn_message('Connected to: ' + idn_str.strip())
    
def on_button_disconnect_clicked(b):
    print_conn_message('')
    disconnect()

btn_connect = widgets.Button(description="Connect")
btn_disconnect = widgets.Button(description="Disconnect")

btn_connect.on_click(on_button_connect_clicked)
btn_disconnect.on_click(on_button_disconnect_clicked)

display(addr_textbox, btn_connect, btn_disconnect, output)

Text(value='', description='Address:', placeholder='IP Address or Slot Id')

Button(description='Connect', style=ButtonStyle())

Button(description='Disconnect', style=ButtonStyle())

Output()

Load TEProteus.dll from : C:\WINDOWS\system32\TEProteus.dll


In [5]:
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# all IQM modes are in 16bit mode
dac_mode = 16


if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEL:ALL')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

Tabor Electronics,P9484M,000002221014,1.223.0
0, no error


## Configure Generator

In [6]:
# set Proteus to interpolation X1. this will move the module to 16bit mode if it was in 8.
inst.send_scpi_cmd(':SOUR:INT X1')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error


#### Create Two segments

In [7]:
# Build two AWG segment one for I and one for Q

cycleLen = 1024
numCycles = 1
segLen = cycleLen * numCycles

# Build waveform (cosine wave)
x = np.linspace(
    start=0, stop=2 * np.pi * numCycles, num=segLen, endpoint=False)

I = (np.cos(x) + 1.0) * half_dac
I = np.round(I)
I = np.clip(I, 0, max_dac)
I = I.astype(data_type)

# download it to segment 1 of channel 1
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEF 1,' + str(segLen))
inst.send_scpi_cmd(':TRAC:SEL 1')
# download the waveform to the selected segment
inst.write_binary_data(':TRAC:DATA', I)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

cycleLen = 1024
numCycles = 1
segLen = cycleLen * numCycles

# Build waveform (sine wave)
x = np.linspace(
    start=0, stop=2 * np.pi * numCycles, num=segLen, endpoint=False)

Q = (np.sin(x) + 1.0) * half_dac
Q = np.round(Q)
Q = np.clip(Q, 0, max_dac)
Q = Q.astype(data_type)

# download it to segment 2 of channel 1
# we must select channel 1 in order to write the segment to the same DDR as channel1.
# in P9082 selecting channel 2 will access DDR2 and not DDR1
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':TRAC:DEF 2,' + str(segLen))
inst.send_scpi_cmd(':TRAC:SEL 2')
# download the waveform to the selected segment
inst.write_binary_data(':TRAC:DATA', Q)
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error
0, no error


#### DAC clock and output

In [8]:
# AWG Channl 1 (output) should be connected to scope input 1
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':SOUR:FUNC:MODE:SEGM 1')
inst.send_scpi_cmd(':OUTP ON')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# AWG Channl 2 (output) should be connected to scope input 2
inst.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':SOUR:FUNC:MODE:SEGM 2')
inst.send_scpi_cmd(':OUTP ON')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# chande DAC clock to 2500MHz
inst.send_scpi_cmd(':SOUR:FREQ 2.5e9')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

0, no error
0, no error
0, no error


#### at this point we should see a sin wave and a cos wave on the scope

#### switching to modulation mode

In [9]:
# set Proteus to interpolation X8.
inst.send_scpi_cmd(':SOUR:INT ' + str(Interpolation))
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# set NCOab frequency of DUC1
inst.send_scpi_cmd(':INST:CHAN 1')
inst.send_scpi_cmd(':SOUR:CFR 100.0e6')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# set NCOab frequency of DUC2 incase Interpolation is X1,X2,X4
inst.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':SOUR:CFR 100.0e6')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# set modulation to HALF
inst.send_scpi_cmd(':SOUR:IQM HALF')
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

# chande DAC clock to 9000Hz
inst.send_scpi_cmd(':SOUR:FREQ ' + str(maxRate))
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)


0, no error
208, syntax error in scpi
208, syntax error in scpi
0, no error
0, no error


In [10]:
inst.send_scpi_cmd(':INST:CHAN 1', paranoia_level=0)

0

## read debug registers

In [12]:
import tempfile
import webbrowser
Debug = False
if Debug == True :
    channb = 1
    cmd = ':INST:CHAN {0}; :SYST:INF:REG?'.format(channb)
    html_str = inst.send_scpi_query(cmd, max_resp_len=200000)
    #print(html_str)
    with tempfile.NamedTemporaryFile('w', delete=False, suffix='.html') as f:
        url = 'file://' + f.name
        f.write(html_str)
    webbrowser.open(url)

## End

In [13]:
# Disconnect
print_conn_message('')
disconnect()